#  "UJ SN2019 Zadanie 2: Nocne Ptasie Wędrówki"

# MLP

In [1]:
import torch
SEED = 1024
torch.manual_seed(SEED)

## Load already prepared train and validation data for second model
This model will classify whether in given second is bird voice or not.

In [2]:
import pathlib

directory = 'data/'
p = pathlib.Path(directory)
if not p.is_dir():
    raise ValueError('Directory: {directory} does not exist. Please, run firstly ConvNet.ipynb for creating data')
    

X_train_classifier = torch.load(directory + 'X_train_classifier.pt')
y_train_classifier = torch.load(directory + 'y_train_classifier.pt')
X_validation_classifier = torch.load(directory + 'X_validation_classifier.pt')
y_validation_classifier = torch.load(directory + 'y_validation_classifier.pt')

In [3]:
from torch.utils.data import TensorDataset, DataLoader

batch_size=64

train_dataset_classifier = TensorDataset(X_train_classifier, y_train_classifier)
validation_dataset_classifier = TensorDataset(X_validation_classifier, y_validation_classifier)
    
train_dataloader_classifier = DataLoader(train_dataset_classifier, batch_size=batch_size, shuffle=True)
validation_dataloader_classifier = DataLoader(validation_dataset_classifier, batch_size=batch_size, shuffle=True)

## MLP model
Input for this model is output from another model (convolutional neural network)

In [4]:
classifier =  torch.nn.Sequential(
    torch.nn.Linear(in_features=10, out_features=300),
    torch.nn.ReLU(),
    torch.nn.Linear(in_features=300, out_features=150),
    torch.nn.ReLU(),
    torch.nn.Linear(in_features=150, out_features=1),
    torch.nn.Sigmoid()
)

In [5]:
import torch.optim as optim
from sklearn.metrics import roc_auc_score

epoch = 30
optimizer_classifier: torch.optim.Optimizer = optim.Adam(classifier.parameters())
criterion_classifier = torch.nn.BCELoss()
best_validation_score_classifier = 0
epochs_without_improvement_classifier = 0
MAX_POSSIBLE_EPOCHS_WITHOUT_IMPROVEMENT_CLASSIFIER = 7


for e in range(epoch):
    preds_second_model_train = []
    y_labels_second_model_train = []
    print(f"\nEPOCH: {e}")
    loss_train_classifier : int = 0
    correct_train_classifier: int = 0
    for i, (x, y) in enumerate(train_dataloader_classifier):
        correct_in_batch_classifier = 0
        optimizer_classifier.zero_grad()
        output_classifier: torch.Tensor = classifier(x)
        loss_classifier: torch.Tensor = criterion_classifier(output_classifier.flatten(), y)
        loss_classifier.backward()
        optimizer_classifier.step()
        loss_train_classifier += loss_classifier.item()
        for predicted, correct in zip(output_classifier.flatten(), y): 
            if predicted >= 0.5 and correct==1:
                correct_in_batch_classifier += 1
            if predicted < 0.5 and correct==0:
                correct_in_batch_classifier += 1
        correct_train_classifier += correct_in_batch_classifier
    print(f"Train accuracy: {correct_train_classifier / len(y_train_classifier)}")
    
    preds_second_model_validation = []
    y_labels_second_model_validation = []
    with torch.no_grad():
        for i, (x, y) in enumerate(validation_dataloader_classifier):
            output_classifier: torch.Tensor = classifier(x)
            preds_second_model_validation.append(output_classifier.flatten())
            y_labels_second_model_validation.append(y)
        preds_second_model_validation = torch.cat(preds_second_model_validation)
        y_labels_second_model_validation = torch.cat(y_labels_second_model_validation)
        validation_score_classifier = roc_auc_score(y_labels_second_model_validation.numpy(), preds_second_model_validation.numpy())
        print(f"Validation ROC AUC score: {validation_score_classifier}")
        if validation_score_classifier > best_validation_score_classifier:
            best_validation_score_classifier = validation_score_classifier
            torch.save(classifier.state_dict(), '../../saved_model/model_classifier.pt')
            epochs_without_improvement_classifier = 0
        else:
            epochs_without_improvement_classifier +=1
        if epochs_without_improvement_classifier == MAX_POSSIBLE_EPOCHS_WITHOUT_IMPROVEMENT_CLASSIFIER:
            break


EPOCH: 0
Train accuracy: 0.7871019427198077
Validation ROC AUC score: 0.7633924884328613

EPOCH: 1
Train accuracy: 0.8417784898858401
Validation ROC AUC score: 0.7605395830734059

EPOCH: 2
Train accuracy: 0.8489885840176247
Validation ROC AUC score: 0.759302474743836

EPOCH: 3
Train accuracy: 0.8527939114760665
Validation ROC AUC score: 0.7594412979796826

EPOCH: 4
Train accuracy: 0.8459843781293811
Validation ROC AUC score: 0.7616097520687297

EPOCH: 5
Train accuracy: 0.844181854596435
Validation ROC AUC score: 0.7620314935447194

EPOCH: 6
Train accuracy: 0.8525936310835169
Validation ROC AUC score: 0.757858888816646

EPOCH: 7
Train accuracy: 0.8563989585419587
Validation ROC AUC score: 0.7610131636058193


## Prepare first model (convolutional neural network)

In [8]:
class BirdDetector(torch.nn.Module):
    
    def __init__(self):
        super(BirdDetector, self).__init__()
        self.pool = torch.nn.MaxPool2d(2)
        
        self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=128, kernel_size=5, padding=2)
        self.bn1 = torch.nn.BatchNorm2d(128)
        
        self.conv2 = torch.nn.Conv2d(in_channels=128, out_channels=96, kernel_size=5, padding=2)
        self.bn2 = torch.nn.BatchNorm2d(96)
        
        self.conv3 = torch.nn.Conv2d(in_channels=96, out_channels=64, kernel_size=3, padding=1)
        self.bn3 = torch.nn.BatchNorm2d(64)
        
        self.fc1 = torch.nn.Linear(448, 100)
        self.fc2 = torch.nn.Linear(100, 1)
        
        self.sigmoid = torch.nn.Sigmoid()
            
    def forward(self, x):
        
        out = self.pool(self.bn1(torch.relu(self.conv1(x))))
        out = self.pool(self.bn2(torch.relu(self.conv2(out))))
        out = self.pool(self.bn3(torch.relu(self.conv3(out))))
        
        out = out.view(out.size(0), -1)
        
        out = self.fc1(out)
        out = self.fc2(out)
        
        return self.sigmoid(out)
    
model = BirdDetector()
model.load_state_dict(torch.load('../../saved_model/model.pt'))

## Load already prepared test data and use it to predict whether in given recording and second is bird voice or not
Get output from first model (convolutional neural network) and us it as input for second model (classifier)

In [6]:
import pathlib
import numpy as np
np.random.seed(SEED)

def load_data(directory, dataset_name):
    p = pathlib.Path(directory)
    if not p.is_dir():
        raise ValueError('Directory: {directory} does not exist. Please, run firstly convolutional_neural_network.ipynb for creating data')
    return np.load(pathlib.Path(directory + dataset_name + '.npy'))


In [9]:
test_data_directory = '../../data/'

test_X = load_data(test_data_directory, 'X_test')
test_X = test_X.reshape(-1, test_X.shape[2], test_X.shape[3])
X_test = torch.from_numpy(test_X).float()
test_dataset = TensorDataset(X_test)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

with torch.no_grad():
    predictions_test = []
    for x in test_dataloader:
        output: torch.Tensor = model(x[0].view(-1, 1, x[0].shape[1], x[0].shape[2]))
        predictions_test.append(output.view(-1))
    predictions_test = torch.cat(predictions_test)
    test_input_for_second_model = []
    for i in range(0, len(predictions_test), 10):
        test_input_for_second_model.append(predictions_test[i : i+10])
    X_test_classifier: torch.Tensor = torch.stack(test_input_for_second_model)
    test_dataset_classifier = TensorDataset(X_test_classifier)
    test_dataloader_classifier = DataLoader(test_dataset_classifier, batch_size=batch_size, shuffle=False)
    
    test_outputs = []
    for x in test_dataloader_classifier:
        output_classifier: torch.Tensor = classifier(x[0])
        test_outputs.append(output_classifier.flatten())
    test_outputs = torch.cat(test_outputs)

## Save predictions for test
Only for recordings and seconds from sampleSubmission.csv

In [10]:
import pandas as pd

def get_indices_from_sample_submission():
    sample_submission_csv = pd.read_csv('../../submission/sampleSubmission.csv')
    return sample_submission_csv['sample_id'].tolist()

def create_dataframe_with_predictions(predictions):
    indices_from_sample_submission = get_indices_from_sample_submission()
    submission_df = pd.DataFrame(columns=['sample_id', 'prediction'])
    for i, element in enumerate(predictions):
        submission_df.loc[i] = [str(int(i/10)+1) + '/' +str(i%10)] + [element]
    return submission_df[submission_df['sample_id'].isin(indices_from_sample_submission)]

In [11]:
my_submission_df = create_dataframe_with_predictions(test_outputs.numpy())
my_submission_df.to_csv('../../submission/mySubmission.csv', index=False)